In [5]:
# See https://pypsa.readthedocs.io/en/latest/user-guide/contingency-analysis.html#branch-outage-distribution-factors-bodf
import polars as pl

from benchmarks.utils import mock_snakemake

DF_CUTOFF = 1e-5


if "snakemake" not in globals():  # noqa: F821
    snakemake = mock_snakemake("compute_bodf")


lines = pl.read_parquet(snakemake.input.lines)
lines = lines.filter(~pl.col("is_leaf")).select("line_id", "from_bus", "to_bus")
ptdf = pl.read_parquet(snakemake.input.ptdf)
lines

line_id,from_bus,to_bus
u32,i64,i64
3,8382,8794
5,8571,8572
6,2668,8730
7,1180,4214
8,308,5065
…,…,…
8119,5543,5550
8120,5548,7643
8121,7850,8488


In [6]:
bptdf = (
    lines.rename({"line_id": "outage"})
    .join(
        ptdf.rename({"injection": "from_bus", "factor": "injection"}),
        on="from_bus",
    )
    .join(
        ptdf.rename({"injection": "to_bus", "factor": "withdrawal"}),
        on=["to_bus", "line"],
    )
    .select(
        "outage",
        "line",
        factor=pl.col("injection") - pl.col("withdrawal"),
    )
)
bptdf

outage,line,factor
u32,i32,f64
1344,2,0.000365
805,2,-0.00003
5927,2,-0.000024
46,2,-0.000005
4629,2,2.4301e-7
…,…,…
5830,8690,-0.000316
4979,8690,-1.1102e-16
1385,8690,-7.9845e-9


In [7]:
diagonal_entries = bptdf.filter(pl.col("outage") == pl.col("line"))
offdiagonal_entries = bptdf.filter(pl.col("outage") != pl.col("line"))
diagonal_entries

outage,line,factor
u32,i32,f64
3,3,-0.000005
5,5,1.7163e-8
6,6,2.7667e-8
7,7,5.5833e-10
8,8,0.000003
…,…,…
8114,8114,3.3127e-10
8116,8116,-1.2989e-8
8121,8121,5.1449e-10


In [9]:
assert diagonal_entries.filter(pl.col("factor") >= (1 - DF_CUTOFF)).height == 0, (
    "Grid is not well connected"
)
# diagonal_entries.filter(pl.col("factor") >= (1 - DF_CUTOFF))

In [10]:
bodf = (
    offdiagonal_entries.join(
        diagonal_entries.select("outage", denominator=1 - pl.col("factor")),
        on="outage",
    )
    .with_columns(pl.col("factor") / pl.col("denominator"))
    .drop("denominator")
)
bodf

outage,line,factor
u32,i32,f64
1344,2,0.000364
805,2,-0.00003
46,2,-0.000005
4629,2,2.4301e-7
5295,2,4.7511e-7
…,…,…
5830,8690,-0.000316
4979,8690,-1.1102e-16
1385,8690,-7.9845e-9


In [11]:
# Filter out near zeros
bodf = bodf.filter(pl.col("factor").abs() > DF_CUTOFF)
bodf

outage,line,factor
u32,i32,f64
1344,2,0.000364
805,2,-0.00003
1135,2,-0.002688
6271,2,0.029677
7934,2,0.008839
…,…,…
7163,8690,-0.000414
1589,8690,-0.000811
6041,8690,-0.000025


In [12]:
bodf.write_parquet(snakemake.output[0])